# Runtime Test

Before runing this test, please download models from [https://nomeroff.net.ua/models/](https://nomeroff.net.ua/models/) to **./models/** directory. 

In [1]:
# Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
#os.environ["CUDA_HOME"] = "/usr/local/cuda" 
# os.environ["CUDA_VISIBLE_DEVICES"] = ""  # For CPU inference

In [2]:
 # dynamically grow the memory used on the GPU
from tensorflow.compat.v1.keras.backend import set_session
import tensorflow as tf
#config = tf.compat.v1.ConfigProto()
#config.gpu_options.allow_growth = True 
#sess = tf.compat.v1.Session(config=config)
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
set_session(sess)


In [ ]:
class MyPipeline(Pipeline):
    def define_graph(self):
        img_reader  = ops.FileReader(file_root = "image_dir", seed = 1)
        img_files, labels = img_reader()  # creates an instance of `FileReader`
        mask_files, _ = mask_reader()     # creates another instance of `FileReader`
        decode = ops.ImageDecoder()
        images = decode(img_files)  # creates an instance of `ImageDecoder`
        masks  = decode(mask_files)   # creates another instance of `ImageDecoder`
        return [images, masks, labels]

pipe = MyPipeline(batch_size = 4, num_threads = 2, device_id = 0)
pipe.build()

In [3]:
# Import all necessary libraries.
import numpy as np
import sys
import glob
import time
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
from turbojpeg import TurboJPEG
jpeg = TurboJPEG()

# NomeroffNet path
NOMEROFF_NET_DIR = os.path.abspath('../')

# specify the path to Mask_RCNN if you placed it outside Nomeroff-net project
CRAFT_DIR = os.path.join(NOMEROFF_NET_DIR, 'CRAFT-pytorch')

sys.path.append(NOMEROFF_NET_DIR)
sys.path.append(CRAFT_DIR)

In [4]:
!pip3 install -U git+git://github.com/lilohuang/PyTurboJPEG.git

  Cloning git://github.com/lilohuang/PyTurboJPEG.git to /tmp/pip-req-build-yz65rkbm
  Created wheel for PyTurboJPEG: filename=PyTurboJPEG-1.4.1-py3-none-any.whl size=7846 sha256=808a3b0c0160f5c67f1d1521e526540416bc9e5511a959d546218e35c11d4896
  Stored in directory: /tmp/pip-ephem-wheel-cache-eeejipbt/wheels/0b/47/e5/13fc5b97f66f9e83aa0c98f39496df95b023ec2c847ef49c0e
Successfully built PyTurboJPEG
  Attempting uninstall: PyTurboJPEG
    Found existing installation: PyTurboJPEG 1.4.1
    Uninstalling PyTurboJPEG-1.4.1:
      Successfully uninstalled PyTurboJPEG-1.4.1


In [5]:
from NomeroffNet.MmdetectionDetector import Detector

detector = Detector()
detector.loadModel(cfg_path = '../models/faster_rcnn_r50_caffe_fpn_mstrain_1x_numberpale.py',
                 checkpoint_file = '../models//epoch_12.pth',
                 devie='cuda:0')

Use load_from_local loader


In [6]:
from NomeroffNet.BBoxNpPoints import NpPointsCraft

npPointsCraft = NpPointsCraft()
npPointsCraft.load()

Loading weights from checkpoint (/var/www/nomeroff-netv2/nomeroff-net/weights/craft_mlt_25k.pth)
Loading weights of refiner from checkpoint (/var/www/nomeroff-netv2/nomeroff-net/weights/craft_refiner_CTW1500.pth)


In [7]:
from NomeroffNet.OptionsDetector import OptionsDetector
from NomeroffNet.TextDetector import TextDetector

# Import license plate recognition tools.
from NomeroffNet.RectDetector import  RectDetector

rectDetector = RectDetector()

optionsDetector = OptionsDetector()
optionsDetector.load("latest")

# Initialize text detector.
textDetector = TextDetector({
    "eu_ua_2004_2015": {
        "for_regions": ["eu_ua_2015", "eu_ua_2004"],
        "model_path": "latest"
    },
    "eu_ua_1995": {
        "for_regions": ["eu_ua_1995"],
        "model_path": "latest"
    },
    "eu": {
        "for_regions": ["eu"],
        "model_path": "latest"
    },
    "ru": {
        "for_regions": ["ru", "eu-ua-fake-lnr", "eu-ua-fake-dnr"],
        "model_path": "latest" 
    },
    "kz": {
        "for_regions": ["kz"],
        "model_path": "latest"
    },
    "ge": {
        "for_regions": ["ge"],
        "model_path": "latest"
    },
    "su": {
        "for_regions": ["su"],
        "model_path": "latest"
    }
})

In [8]:
async def test(dirName, fname, min_bbox_acc = 0.5):
    start_time = time.time()
    img_path = os.path.join(dirName, fname)
    with open(img_path, 'rb') as in_file:
        img = jpeg.decode(in_file.read())
    #img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    image_load_time = time.time() - start_time

    start_time = time.time()
    targetBoxes = detector.detect_bbox([img], min_bbox_acc=min_bbox_acc, max_img_w=800)[0]
    detect_bbox_time = time.time() - start_time
    
    start_time = time.time()
    all_points = npPointsCraft.detect(img, targetBoxes)
    all_points = [ps for ps in all_points if len(ps)]
    craft_time = time.time() - start_time
    
    start_time = time.time()
    zones = rectDetector.get_cv_zonesBGR(img, all_points)
    perspective_align_time = time.time() - start_time
    
    start_time = time.time()
    regionIds, stateIds, countLines = optionsDetector.predict(zones)
    regionNames = optionsDetector.getRegionLabels(regionIds)
    classification_time = time.time() - start_time
    
    start_time = time.time()
    textArr = textDetector.predict(zones, regionNames, countLines)
    ocr_time = time.time() - start_time
    return (image_load_time, detect_bbox_time, craft_time, perspective_align_time, classification_time, ocr_time)

In [9]:
N = 10

i = 0
j = 0

image_load_time_all        = 0
detect_bbox_time_all       = 0
craft_time_all             = 0
perspective_align_time_all = 0
classification_time_all    = 0
ocr_time_all               = 0

start_time = time.time()
rootDir = 'images/'
for i in np.arange(N):
    print("pass {}".format(i))
    for dirName, subdirList, fileList in os.walk(rootDir):
        for fname in fileList:
            image_load_time, \
                detect_bbox_time, \
                craft_time, \
                perspective_align_time, \
                classification_time, \
                ocr_time = await test(dirName, fname)
            image_load_time_all        += image_load_time
            detect_bbox_time_all       += detect_bbox_time
            craft_time_all             += craft_time
            perspective_align_time_all += perspective_align_time
            classification_time_all    += classification_time
            ocr_time_all               += ocr_time
            j += 1
            #print(i, j)
    i += 1
end_time = time.time() - start_time

pass 0


/var/www/mmdetection/mmdet/datasets/utils.py:62: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(
/var/www/nomeroff-netv2/nomeroff-net/CRAFT-pytorch/craft_utils.py:239: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  polys = np.array(polys)


pass 1
pass 2
pass 3
pass 4
pass 5
pass 6
pass 7
pass 8
pass 9


### Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz + NVIDIA RTX 2080TI

In [10]:
print(f"Processed {j} photos")
print(f"Time {end_time}")
print(f"One photo process {end_time/j} seconds")
print()
print(f"image_load_time_all {image_load_time_all}; {image_load_time_all/j} per one photo")
print(f"detect_bbox_time_all {detect_bbox_time_all}; {detect_bbox_time_all/j} per one photo")
print(f"craft_time_all {craft_time_all}; {craft_time_all/j} per one photo")
print(f"perspective_align_time_all {perspective_align_time_all}; {perspective_align_time_all/j} per one photo")
print(f"classification_time_all {classification_time_all}; {classification_time_all/j} per one photo")
print(f"ocr_time_all {ocr_time_all}; {ocr_time_all/j} per one photo")

Processed 260 photos
Time 36.345908641815186
One photo process 0.13979195631467378 seconds

image_load_time_all 7.936519384384155; 0.030525074555323675 per one photo
detect_bbox_time_all 13.709846496582031; 0.05273017883300781 per one photo
craft_time_all 4.996851921081543; 0.019218661234929012 per one photo
perspective_align_time_all 0.12414097785949707; 0.00047746529945960414 per one photo
classification_time_all 2.6096596717834473; 0.01003715258378249 per one photo
ocr_time_all 6.927104473114014; 0.026642709511976975 per one photo


In [11]:
# keras classification                             time 0.0423
# keras classification with bach size              time 0.0347
# keras classification model()                     time 0.0120
# keras classification model() tr=false            time 0.0120
# keras classification model() tr=false, one batch time 0.0100

In [12]:
# keras ocr                                        time 0.0562
# keras ocr with bach size                         time 0.0537
# keras classification model()                     time 0.0307
# keras classification model() tr=false            time 0.0295
# keras classification model() tr=false, one batch time 0.0284

In [13]:
# cv2 image load time 0.057034785930926986
# image load time 0.030525074555323675

### Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz + Quadro RTX 8000